In [1]:
import os
import sys
import numpy as np
import pandas as pd
from Utils.forest_cover import get_cover,green_cover_plot, green_cover_plot_smooth
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image, ImageFilter
import imutils
from pyimagesearch.colorlabeler import ColorLabeler
from Utils.read_coord import kml_coord
from Utils.sentinal_request import senti_api,bbox_cord

C:\Users\DELL\anaconda3\envs\sentinal\lib\site-packages\fastkml\config.py:28: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")
C:\Users\DELL\anaconda3\envs\sentinal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
file_path = input('Enter kml file path: ')
print(file_path)
#C:\Users\Harith\EngenuityAI\SPE\SPE-development\sentinal\gAMPOLA TEST SITE 2.kml
if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()

        #print(lines)
else:
    print('The specified file does NOT exist')



# getting polygon coordinates
poly_coord = kml_coord(file_path)
#test_kml_files/Kolonnawa.kml

Enter kml file path: test_kml_files/Kolonnawa.kml
test_kml_files/Kolonnawa.kml
The file exists


In [9]:
#for original
bbx_coord,poly = bbox_cord(poly_coord)

In [2]:
img = cv2.imread('output/gren_cover_no_legend.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [4]:
#define colours

#green color 
lower_green = np.array([40,70,80])
upper_green = np.array([70,255,255])

#white
lower_white = np.array([255,255,255])
upper_white = np.array([255,255,255])


#add col mask
green = cv2.inRange(hsv, lower_green, upper_green)
white = cv2.inRange(img, lower_white, upper_white)#use bgr instead of img

In [5]:
def get_contours(img,hsv):
    #add col mask
    green = cv2.inRange(hsv, lower_green, upper_green)
    white = cv2.inRange(img, lower_white, upper_white)#use bgr instead of img
    #gree conoturs
    cnts_green = cv2.findContours(green,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts_green = imutils.grab_contours(cnts_green)
    #white contours
    cnts_white = cv2.findContours(white,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts_white = imutils.grab_contours(cnts_white)
    
    return cnts_green,cnts_white[1] #the boundry contour

In [16]:
def point_to_coord(cnt,bbx_coord):
    poly_arr = np.squeeze(cnt,1)
    #get the bbox
    df = pd.DataFrame(poly_arr, columns = ['lat','long'])
    Xmin = df['lat'].min()
    Ymin = df['long'].min()
    Xmax = df['lat'].max()
    Ymax = df['long'].max()
    bbox = [Xmin,Ymin,Xmax,Ymax]
    
    #coordinate lengths
    coord_len = bbx_coord[2] - bbx_coord[0]
    coord_h = bbx_coord[3] - bbx_coord[1]
    
    #array point lengths
    img_len = bbox[2] - bbox[0]
    img_h = bbox[3] - bbox[1]
    
    #calculate ration diff
    ratio_len = coord_len/img_len
    ratio_h = coord_h/img_h
    
    #get the conversion 
    #coord = point min + (point-point min) * ratio
    lat = []
    long = []
    for index, row in df.iterrows():
        x = (row['lat'] - bbox[0]) * ratio_len
        y = (row['long'] - bbox[1]) * ratio_len
        #print(x)
        lat.append(bbx_coord[0]+x)
        long.append(bbx_coord[1]+y)
        
    poly = list(zip(lat, long))
    return poly

In [7]:
greens,white = get_contours(img,hsv)

In [17]:
boundry_poly = point_to_coord(white,bbx_coord)
boundry_poly

[(79.88636855615572, 6.915872379284323),
 (79.8864204913662, 6.915820444073838),
 (79.88657629699766, 6.915820444073838),
 (79.88662823220814, 6.915872379284323),
 (79.88662823220814, 6.915976249705294),
 (79.88688790826058, 6.915976249705294),
 (79.88693984347105, 6.916028184915779),
 (79.887147584313, 6.916028184915779),
 (79.88719951952348, 6.9160801201262645),
 (79.88719951952348, 6.91613205533675),
 (79.88740726036542, 6.91613205533675),
 (79.88745919557591, 6.9161839905472355),
 (79.88745919557591, 6.916235925757721),
 (79.88766693641784, 6.916235925757721),
 (79.88771887162834, 6.916287860968206),
 (79.88771887162834, 6.916339796178691),
 (79.88792661247028, 6.916339796178691),
 (79.88797854768076, 6.916391731389177),
 (79.88813435331221, 6.916391731389177),
 (79.8881862885227, 6.916443666599662),
 (79.8881862885227, 6.916495601810147),
 (79.88839402936465, 6.916495601810147),
 (79.88844596457513, 6.916547537020632),
 (79.88844596457513, 6.916599472231118),
 (79.88865370541707, 

In [18]:
green_coords = []
for green in greens:
    print(green.shape)
    g_coord = point_to_coord(green,bbx_coord)
    green_coords.append(g_coord)

(4, 1, 2)
(3, 1, 2)
(3, 1, 2)
(4, 1, 2)
(2, 1, 2)
(20, 1, 2)
(43, 1, 2)
(10, 1, 2)
(1, 1, 2)
(20, 1, 2)
(6, 1, 2)
(4, 1, 2)
(2, 1, 2)
(4, 1, 2)
(2, 1, 2)
(11, 1, 2)
(2, 1, 2)
(5, 1, 2)
(14, 1, 2)
(17, 1, 2)
(2, 1, 2)
(2, 1, 2)
(4, 1, 2)
(2, 1, 2)
(4, 1, 2)
(4, 1, 2)
(36, 1, 2)
(1, 1, 2)
(14, 1, 2)
(13, 1, 2)
(4, 1, 2)
(4, 1, 2)
(1, 1, 2)
(48, 1, 2)
(3, 1, 2)
(4, 1, 2)
(15, 1, 2)
(3, 1, 2)
(71, 1, 2)
(1, 1, 2)
(2, 1, 2)
(2, 1, 2)
(2, 1, 2)
(2, 1, 2)
(1, 1, 2)
(2, 1, 2)
(2, 1, 2)
(3, 1, 2)
(9, 1, 2)
(4, 1, 2)
(2, 1, 2)
(2, 1, 2)
(7, 1, 2)
(2, 1, 2)
(4, 1, 2)
(8, 1, 2)
(4, 1, 2)
(3, 1, 2)
(5, 1, 2)
(7, 1, 2)
(8, 1, 2)
(11, 1, 2)
(2, 1, 2)
(2, 1, 2)
(8, 1, 2)
(3, 1, 2)
(16, 1, 2)
(4, 1, 2)
(2, 1, 2)
(78, 1, 2)
(3, 1, 2)
(12, 1, 2)
(5, 1, 2)
(3, 1, 2)
(2, 1, 2)
(9, 1, 2)
(3, 1, 2)
(2, 1, 2)
(5, 1, 2)
(4, 1, 2)
(4, 1, 2)
(5, 1, 2)
(4, 1, 2)
(4, 1, 2)
(2, 1, 2)
(2, 1, 2)
(6, 1, 2)
(4, 1, 2)
(4, 1, 2)
(81, 1, 2)
(22, 1, 2)
(6, 1, 2)
(11, 1, 2)
(12, 1, 2)
(69, 1, 2)
(6, 1, 2)
(6, 1, 2)
(3, 1, 2

C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  ratio_h = coord_h/img_h
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  ratio_len = coord_len/img_len
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  ratio_h = coord_h/img_h
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:28: RuntimeWarning: invalid value encountered in double_scalars
  x = (row['lat'] - bbox[0]) * ratio_len
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:29: RuntimeWarning: invalid value encountered in double_scalars
  y = (row['long'] - bbox[1]) * ratio_len
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  ratio_len = coord_len/img_len
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076

(11, 1, 2)
(9, 1, 2)
(6, 1, 2)
(6, 1, 2)
(14, 1, 2)
(4, 1, 2)
(6, 1, 2)
(4, 1, 2)
(9, 1, 2)
(4, 1, 2)
(1, 1, 2)
(11, 1, 2)
(2, 1, 2)
(2, 1, 2)
(3, 1, 2)
(4, 1, 2)
(7, 1, 2)
(9, 1, 2)
(3, 1, 2)
(4, 1, 2)
(2, 1, 2)
(3, 1, 2)
(4, 1, 2)
(2, 1, 2)
(4, 1, 2)
(3, 1, 2)
(16, 1, 2)
(2, 1, 2)
(8, 1, 2)
(1, 1, 2)
(6, 1, 2)
(1, 1, 2)
(2, 1, 2)
(8, 1, 2)
(3, 1, 2)
(3, 1, 2)
(4, 1, 2)
(4, 1, 2)
(9, 1, 2)
(4, 1, 2)
(2, 1, 2)
(2, 1, 2)
(2, 1, 2)
(4, 1, 2)
(2, 1, 2)
(2, 1, 2)
(2, 1, 2)
(10, 1, 2)
(2, 1, 2)
(2, 1, 2)
(10, 1, 2)
(6, 1, 2)
(3, 1, 2)
(2, 1, 2)
(17, 1, 2)
(3, 1, 2)
(13, 1, 2)
(67, 1, 2)
(8, 1, 2)
(6, 1, 2)
(1, 1, 2)
(12, 1, 2)
(2, 1, 2)
(8, 1, 2)
(19, 1, 2)
(2, 1, 2)
(5, 1, 2)
(4, 1, 2)
(9, 1, 2)
(8, 1, 2)
(4, 1, 2)
(2, 1, 2)
(3, 1, 2)
(2, 1, 2)
(4, 1, 2)
(6, 1, 2)
(17, 1, 2)
(11, 1, 2)
(24, 1, 2)
(3, 1, 2)
(6, 1, 2)
(11, 1, 2)
(2, 1, 2)
(6, 1, 2)
(5, 1, 2)
(2, 1, 2)
(4, 1, 2)
(1, 1, 2)
(2, 1, 2)
(2, 1, 2)
(12, 1, 2)
(7, 1, 2)
(4, 1, 2)
(2, 1, 2)
(4, 1, 2)
(20, 1, 2)
(4, 1, 2)
(31, 1, 2)
(1

C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  ratio_len = coord_len/img_len
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  ratio_h = coord_h/img_h
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:28: RuntimeWarning: invalid value encountered in double_scalars
  x = (row['lat'] - bbox[0]) * ratio_len
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:29: RuntimeWarning: invalid value encountered in double_scalars
  y = (row['long'] - bbox[1]) * ratio_len
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  ratio_h = coord_h/img_h
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\621786074.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  ratio_h = coord_h/img_h
C:\Users\DELL\AppData\Local\Temp\ipykernel_4076\62178

In [22]:
green_coords

[[(79.91664678386864, 6.915820444073838),
  (79.88636855615572, 6.946098671786765),
  (79.88636855615572, 7.036933354925545),
  (79.91664678386864, 7.036933354925545)],
 [(79.88636855615572, 6.915820444073838),
  (79.88636855615572, 6.946098671786765),
  (79.91664678386864, 6.915820444073838)],
 [(79.91664678386864, 6.915820444073838),
  (79.88636855615572, 6.946098671786765),
  (79.91664678386864, 6.946098671786765)],
 [(79.88636855615572, 6.915820444073838),
  (79.88636855615572, 6.946098671786765),
  (79.91664678386864, 6.946098671786765),
  (79.91664678386864, 6.915820444073838)],
 [(79.88636855615572, 6.915820444073838),
  (79.91664678386864, 6.915820444073838)],
 [(79.90907722694041, 6.915820444073838),
  (79.9052924484763, 6.919605222537954),
  (79.9052924484763, 6.938529114858533),
  (79.90150767001218, 6.942313893322649),
  (79.88636855615572, 6.942313893322649),
  (79.88636855615572, 6.949883450250881),
  (79.89015333461984, 6.949883450250881),
  (79.89393811308395, 6.9536682

In [23]:
from Utils.map_view import map_show

In [24]:
map_show(boundry_poly)